In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0, '/data/programs/probe_position_correction_w_ptychonn/pppc')
sys.path.insert(0, '/data/programs/probe_position_correction_w_ptychonn')
import numpy as np
import torch
import matplotlib.pyplot as plt

import pppc
from pppc.configs import PtychoNNTrainingConfigDict
from pppc.ptychonn.trainer import PtychoNNTrainer
from pppc.ptychonn.dataset_handle import HDF5Dataset

[2023-11-21 17:19:35,367] Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[2023-11-21 17:19:35,370] NumExpr defaulting to 8 threads.


In [3]:
config_dict = PtychoNNTrainingConfigDict()
config_dict['batch_size_per_process'] = 64
config_dict['num_epochs'] = 60
config_dict['learning_rate_per_process'] = 1e-3
config_dict['optimizer'] = 'adam'
config_dict['model_save_dir'] = '../../trained_models/model_36SpiralDatasets'
config_dict['validation_ratio'] = 0.01
dataset = HDF5Dataset('data/data_train.h5', verbose=False)
config_dict['dataset'] = dataset
config_dict['debug'] = False

Check some data distributions. 

In [ ]:
def plot_distribution(x):
    x = x.reshape(-1)
    x = x[x > 0]
    plt.hist(np.log10(x), bins=32)
    plt.show()

x, y_mag, y_ph = dataset.__getitems__(list(range(100)))
plot_distribution(x.numpy())
plot_distribution(y_mag.numpy())
plot_distribution(y_ph.numpy())

In [ ]:
trainer = PtychoNNTrainer(config_dict)
trainer.build()
trainer.run_training()

[2023-11-21 17:19:38,860] Using DataParallel with 2 devices.
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 268/268 [04:18<00:00,  1.04it/s]
[2023-11-21 17:23:59,635] Saving improved model after Val Loss improved from inf to 0.54044
[2023-11-21 17:23:59,658] Epoch: 0 | FT  | Train Loss: 0.63818 | Val Loss: 0.54044
[2023-11-21 17:23:59,659] Epoch: 0 | Amp | Train Loss: 0.0816 | Val Loss: 0.0253
[2023-11-21 17:23:59,659] Epoch: 0 | Ph  | Train Loss: 0.557 | Val Loss: 0.515
[2023-11-21 17:23:59,660] Epoch: 0 | Ending LR: 0.000503 
 15%|██████████████████████████████████▉                                                                                                                                                                                                       | 40/268 [00:37<03:43,  1.02it/s]

In [ ]:
trainer.plot_training_history()